In [33]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd

In [12]:
matches_path = 'https://www.vlr.gg/matches/results'
chromedriver_path = r'C:\Users\work\Sportsbetting\Valorant\chromedriver.exe'

In [18]:
every_day_container_class = 'col.mod-1'

In [118]:
# create a new Chrome browser instance
driver = webdriver.Chrome(executable_path=chromedriver_path)

# navigate to a website
driver.get(matches_path)

anchors = driver.find_elements(by=By.CSS_SELECTOR, value='a')

links = []
# Iterate over each anchor element and extract the href attribute
for anchor in anchors:
    href = anchor.get_attribute('href')
    links.append(href)

# container = driver.find_element(by=By.CLASS_NAME, value=every_day_container_class)
# nested_divs = container.find_elements(by=By.CSS_SELECTOR, value='div')
#
# data = []
# for div in nested_divs:
#     data.append((div.text, div.href))

driver.quit()

C:\Users\work\AppData\Local\Temp\ipykernel_16744\675339469.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chromedriver_path)


In [119]:
def get_totals(cell):
    return cell.split()[0]

def get_totals_d(cell):
    return cell.split()[1]

def format_dataframe(dataframes):
    """
    unduplicates data

    :param dataframes: dataframes returned using the pandas read_html function on the page source
    :return: formatted df
    """
    formatting_df = pd.concat(dataframes[:2])

    formatting_df = formatting_df.reset_index()
    formatting_df = formatting_df.drop(columns=['index', 'Unnamed: 1'])

    formatting_df.iloc[:, 1:4] = formatting_df.iloc[:, 1:4].applymap(get_totals)
    formatting_df.iloc[:, 5:] = formatting_df.iloc[:, 5:].applymap(get_totals)
    formatting_df['PLAYER'] = formatting_df['Unnamed: 0'].apply(lambda x: x.split()[0])
    formatting_df['TEAM'] = formatting_df['Unnamed: 0'].apply(lambda x: x.split()[1])

    formatting_df = formatting_df.drop(columns=['Unnamed: 0'])

    formatting_df.iloc[:, 3] = formatting_df.iloc[:, 3].apply(get_totals_d)

    return formatting_df


def get_match_results_data(driver, link):
    """
    :param driver: chromedriver
    :param link: link for particular match results page
    :return: multiple dataframes of match results, names for images/agent names
    """

    driver.get(link)

    source = driver.page_source

    dfs = pd.read_html(source)

    img_elements = driver.find_elements(By.TAG_NAME, value='img')

    # Loop through each image element and extract the "alt" text
    alt_texts = []
    for img_element in img_elements:
        alt_text = img_element.get_attribute('alt')
        if alt_text:
            alt_texts.append(alt_text)


    return dfs, alt_texts

In [ ]:

dfs, alt_texts = get_match_results_data(driver, links[0])

In [ ]:
df1 = format_dataframe(dfs)

In [112]:
df1['AGENT'] = alt_texts[2:12]

In [113]:
df1

,R,ACS,K,D,A,+/–,KAST,ADR,HS%,FK,FD,+/–.1,PLAYER,TEAM,AGENT
0,1.53,249,16,6,3,+10,88%,157,29%,4,0,+4,kamo,GTZ,jett
1,1.53,289,17,8,5,+9,94%,186,39%,2,3,-1,OLIZERA,GTZ,phoenix
2,1.44,243,14,8,10,+6,88%,146,24%,2,1,+1,RobbieBk,GTZ,fade
3,1.25,249,16,10,4,+6,88%,172,27%,1,0,+1,k1zpawn,GTZ,killjoy
4,1.15,183,10,8,9,+2,69%,111,19%,1,2,-1,starkk,GTZ,astra
5,1.21,283,18,14,1,+4,63%,156,28%,3,0,+3,Caim,VK,jett
6,0.57,145,5,15,10,-10,75%,115,19%,1,0,+1,Pedro,VK,skye
7,0.56,138,7,15,2,-8,63%,96,50%,2,2,0,Saa,VK,viper
8,0.41,108,5,14,6,-9,50%,72,15%,0,4,-4,RaykiNN,VK,harbor
9,0.34,101,5,15,1,-10,31%,74,18%,0,4,-4,Tigax,VK,killjoy
